# Homework 9
The goal of this HW is simple. Duplicate homework 8 problem 2 using neural networks. Contrast regression versus using i.) a neural network with 2 hidden nodes and ii.) 4 hidden nodes over 2 layers with 2 hidden nodes in each layer. All layers should be full connectected, MSE the loss function, a linear output activation layer and RELUs otherwise.

You should return an ipynb nothebook with your code and a final table contrasting the different error rates of the different approaches you tried.

----------------------------------------------------------------------------------------------

## Create the dataframe

In [51]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# define the lag function
def lag(a, l):    
    b1 = a[:-l]
    b2 = [np.nan] * l
    b = np.concatenate((b2, b1))
    return b 

# Get the country-specific covid case data
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

# Country country A, Germany
# Select Germany as the picked country A, then remove unnecessary columns
df = df.loc[df['Country/Region'] == 'Germany'].reset_index(drop=True)
df.drop(['Province/State', 'Country/Region', 'Lat', 'Long'], inplace=True, axis=1)
# remove the leading zero
df = df.loc[:, (df != 0).all()]
# convert to array, and reshape it as one vector
arr = np.array(df).reshape((len(df.T),))
y_l5 = arr[5:]
y_l4 = lag(arr, 1)[5:]
y_l3 = lag(arr, 2)[5:]
y_l2 = lag(arr, 3)[5:]
y_l1 = lag(arr, 4)[5:]
y = lag(arr, 5)[5:]
# create the new and main dataframe for the regression analysis
columns = ["Y", "Y_L1", "Y_L2", "Y_L3", "Y_L4", "Y_L5"]
data = np.stack((y, y_l1, y_l2, y_l3, y_l4, y_l5), axis=1)
df_A = pd.DataFrame(data=data, columns=columns)
df_A.head(10)

,Y,Y_L1,Y_L2,Y_L3,Y_L4,Y_L5
0,1.0,4.0,4.0,4.0,5.0,8.0
1,4.0,4.0,4.0,5.0,8.0,10.0
2,4.0,4.0,5.0,8.0,10.0,12.0
3,4.0,5.0,8.0,10.0,12.0,12.0
4,5.0,8.0,10.0,12.0,12.0,12.0
5,8.0,10.0,12.0,12.0,12.0,12.0
6,10.0,12.0,12.0,12.0,12.0,13.0
7,12.0,12.0,12.0,12.0,13.0,13.0
8,12.0,12.0,12.0,13.0,13.0,14.0
9,12.0,12.0,13.0,13.0,14.0,14.0


## Prepare the data 
### Considering two scenarios; tarining on (1) whole data and (2) half data

In [52]:
# whole training data
X = np.array(df_A[['Y_L1', 'Y_L2', 'Y_L3', 'Y_L4', 'Y_L5']])
y = np.array(df_A['Y'])
X = torch.from_numpy(X).float()
y = torch.log(torch.from_numpy(y)).float()
print(X.shape, y.shape)

# half training data
idx_half = round(len(df_A)/2) # get the index for the half split of X
X_half = np.array(df_A[['Y_L1', 'Y_L2', 'Y_L3', 'Y_L4', 'Y_L5']])[:idx_half]
y_half = np.array(df_A['Y'])[:idx_half]
X_half = torch.from_numpy(X_half).float()
y_half = torch.log(torch.from_numpy(y_half)).float()
print(X_half.shape, y_half.shape)

torch.Size([828, 5]) torch.Size([828])
torch.Size([414, 5]) torch.Size([414])


## i.) A neural network with 2 hidden nodes

In [53]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 1000, 5, 2, 1

# Use the nn package to define our model as a sequence of layers.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# Define the loss function
loss_fn = torch.nn.MSELoss(reduction='sum')

# Run the model on whole data
learning_rate = 1e-7
for t in range(10000):
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    if t % 1000 == 0:
        print(t, loss.item())
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
            
## try prediction
y_pred = model(X)
a = y_pred.detach().numpy()

#plt.scatter(a[:,0], y_pred[:,0])            

/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([828])) that is different to the input size (torch.Size([828, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0 5.677257921277198e+18
1000 6161851.5
2000 6161851.5
3000 6161851.5
4000 6161851.5
5000 6161851.5
6000 6161851.5
7000 6161851.5
8000 6161851.5
9000 6161851.5


In [57]:
# Run the model on half data
learning_rate = 1e-7
for t in range(10000):
    y_pred = model(X_half)
    loss = loss_fn(y_pred, y_half)
    if t % 1000 == 0:
        print(t, loss.item())
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
            
## try prediction
y_pred = model(X)
a = y_pred.detach().numpy()  

0 1789253.25
1000 1789253.25
2000 1789253.25
3000 1789253.25
4000 1789253.25
5000 1789253.25
6000 1789253.25
7000 1789253.25
8000 1789253.25
9000 1789253.25


## ii.) 4 hidden nodes over 2 layers with 2 hidden nodes in each layer

In [59]:
# N is batch size; D_in is input dimension;
# H1 is hidden layer 1 dimension; H2 is hidden layer 2 dimension; 
# D_out is output dimension.
N, D_in, H1, H2, D_out = 1000, 5, 2, 2, 1

# Use the nn package to define our model as a sequence of layers.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, D_out),
)

# Define the loss function
loss_fn = torch.nn.MSELoss(reduction='sum')

# Run the model on whole data
learning_rate = 1e-7
for t in range(10000):
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    if t % 1000 == 0:
        print(t, loss.item())
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
            
## try prediction
y_pred = model(X)
a = y_pred.detach().numpy()

#plt.scatter(a[:,0], y_pred[:,0]) 

0 2.8067342843904e+16
1000 6161851.5
2000 6161851.5
3000 6161851.5
4000 6161851.5
5000 6161851.5
6000 6161851.5
7000 6161851.5
8000 6161851.5
9000 6161851.5


In [60]:
# Run the model on half data
learning_rate = 1e-7
for t in range(10000):
    y_pred = model(X_half)
    loss = loss_fn(y_pred, y_half)
    if t % 1000 == 0:
        print(t, loss.item())
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
            
## try prediction
y_pred = model(X)
a = y_pred.detach().numpy()  

0 2398440.25
1000 1789253.25
2000 1789253.25
3000 1789253.25
4000 1789253.25
5000 1789253.25
6000 1789253.25
7000 1789253.25
8000 1789253.25
9000 1789253.25
